In [4]:
import pandas as pd
import os
import folium
import geohash

In [2]:
df = pd.read_csv("results/first_million.csv")

In [3]:
df.head()

,captured time,latitude,longitude,value
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0


In [6]:
def compute_geohash(row):
    return geohash.encode(row["latitude"], row["longitude"], precision=3)

In [7]:
df["geohash"] = df.apply(compute_geohash, axis=1)

In [8]:
df.head()

,captured time,latitude,longitude,value,geohash
0,2018-10-21 01:00:26.000000,36.041080,140.226816,23.0,xn7
1,2018-10-21 01:00:22.000000,37.796306,140.514413,19.0,xne
2,2018-10-21 01:00:16.000000,37.723330,140.476797,15.0,xne
3,2018-10-21 00:59:16.000000,52.444900,13.315000,16.0,u33
4,2018-10-21 01:00:16.000000,37.787500,140.552400,18.0,xne


In [9]:
grouped = df[["geohash", "value"]].groupby(["geohash"]).mean()

In [10]:
grouped.head()

,value
geohash,
8e8,30.033472
9ex,1507.435186
9mu,34.157321
9pp,36.000000
9q4,36.587536


In [17]:
grouped = grouped.reset_index()

In [18]:
grouped.head()

,geohash,value
0,8e8,30.033472
1,9ex,1507.435186
2,9mu,34.157321
3,9pp,36.000000
4,9q4,36.587536


In [19]:
def decode_geohash(row):
    return pd.Series(geohash.decode(row.geohash))

In [23]:
grouped[['latitude', 'longitude']] = grouped.apply(decode_geohash, axis=1)

In [24]:
grouped.head()

,geohash,value,latitude,longitude
0,8e8,30.033472,20.390625,-156.796875
1,9ex,1507.435186,20.390625,-101.953125
2,9mu,34.157321,33.046875,-117.421875
3,9pp,36.000000,40.078125,-124.453125
4,9q4,36.587536,34.453125,-120.234375


In [25]:
len(grouped.index)

213

In [26]:
import folium
from folium.plugins import HeatMap

In [28]:
max_value = grouped.value.max()
max_value

1507.435186035829

In [30]:
hmap = folium.Map(location=[42.5, -75.5], zoom_start=7)

In [35]:
hm_wide = HeatMap(list(zip(grouped.latitude.values, grouped.longitude.values, grouped.value.values)),
                   min_opacity=0.2,
                   max_val=max_value,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )
hmap.add_child(hm_wide)

In [36]:
fukushima = pd.read_csv("results/fukushima_grouped.csv")

In [37]:
fukushima.head()

,geohash,value
0,xnecec,29.000000
1,xnecee,37.000000
2,xnecef,32.714286
3,xneceq,40.807692
4,xnecer,29.800000


In [38]:
fukushima[['latitude', 'longitude']] = fukushima.apply(decode_geohash, axis=1)

In [39]:
len(fukushima.index)

475

In [43]:
hmap = folium.Map(location=[42.5, -75.5], zoom_start=7)
max_value = fukushima.value.max()
hm_wide = HeatMap(list(zip(fukushima.latitude.values, fukushima.longitude.values, fukushima.value.values)),
                   min_opacity=0.2,
                   max_val=max_value,
                   radius=25, blur=25, 
                   max_zoom=1, 
                 )
hmap.add_child(hm_wide)

In [44]:
points = pd.read_csv("results/fukushima_filtered.csv")

In [45]:
points.head()

,captured time,latitude,longitude,value
0,2018-10-21 01:00:22,37.796306,140.514413,19.0
1,2018-10-21 01:00:16,37.723330,140.476797,15.0
2,2018-10-21 01:00:16,37.787500,140.552400,18.0
3,2018-10-21 01:00:16,37.796306,140.514413,61.0
4,2018-10-21 01:00:14,37.792805,140.930957,13.0


In [46]:
len(points)

216452

In [ ]:
hmap = folium.Map(location=[42.5, -75.5], zoom_start=7)
max_value = points.value.max()
hm_wide = HeatMap(list(zip(points.latitude.values, points.longitude.values, points.value.values)),
                   min_opacity=0.2,
                   max_val=max_value,
                   radius=25, blur=25, 
                   max_zoom=1, 
                 )
hmap.add_child(hm_wide)

In [48]:
hmap.save("the_best_heatmap.html")